In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from tools import query_vector_db_articles
title = "Targeted therapy in advanced non-small cell lung cancer: current advances and future trends."
title2 = "Low titers of SARS-CoV-2 neutralizing antibodies after first vaccination dose in cancer patients receiving checkpoint inhibitors."

/home/thomas/anaconda3/envs/takehome/lib/python3.12/site-packages/pinecone/data/index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm
/home/thomas/anaconda3/envs/takehome/lib/python3.12/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [3]:
## CODE TO DELETE PINECONE:
import os
from pinecone.grpc import PineconeGRPC as Pinecone

pc = Pinecone(api_key=os.getenv("PINECONE_API_KEY"))
index = pc.Index("pubmed")

index.delete(delete_all=True)

In [4]:
article_title = "Targeted therapy in advanced non-small cell lung cancer: current advances and future trends."
query = "What disease are they discussing in the paper?"
query2 = "Can you list the protein targets they highlight in the paper?"
query3= "Summarize the main takeaways from the paper."

In [5]:
# completion llm  
from langchain_cohere import ChatCohere

llm = ChatCohere(model="command-r-plus",temperature=0.0,max_tokens=4096)

In [6]:
from langchain.agents import AgentExecutor
from langchain_cohere.react_multi_hop.agent import create_cohere_react_agent
from langchain_core.prompts import ChatPromptTemplate
from langchain.agents import AgentExecutor, create_react_agent

# from langchain.tools.retriever import create_retriever_tool

# vectorstore_search = create_retriever_tool(
#    retriever=retriever,
#    name="vectorstore_search",
#    description="Retrieve relevant info from a vectorstore that contains information from the article. You need to filter with the following metadata when you query the vector database: 'source': 'name_of_the_article' "
# )

# Preamble
preamble = """
You are an expert who answers the user's question with the most relevant datasource.
You are equipped with a special vectorstore of information about scientific articles.
"""

# Prompt template
prompt = ChatPromptTemplate.from_template("{input}")
# Create the ReAct agent
agent = create_cohere_react_agent(
   llm=llm,
   tools=[query_vector_db_articles],
   prompt=prompt,
)

agent_executor = AgentExecutor(agent=agent,
                               tools=[query_vector_db_articles],
                               verbose=True,
                               handle_parsing_errors=True,)


In [8]:
response = agent_executor.invoke({
   "input": "Summarize the main findings of the paper '{}'?".format(article_title),
   "preamble": preamble,
})



> Entering new AgentExecutor chain...

I will search for the paper 'Targeted therapy in advanced non-small cell lung cancer: current advances and future trends' and summarise its main findings.
{'tool_name': 'query_vector_db_articles', 'parameters': {'query': 'Targeted therapy in advanced non-small cell lung cancer: current advances and future trends', 'metadata': {'title': 'Targeted therapy in advanced non-small cell lung cancer: current advances and future trends'}}}
[Document(page_content='Majeed\xa0 et\xa0al. J Hematol Oncol          (2021) 14:108  \nhttps://doi.org/10.1186/s13045-021-01121-2\nREVIEW\nTargeted therapy in\xa0advanced non-small cell \nlung cancer: current advances and\xa0future trends\nUmair Majeed, Rami Manochakian, Yujie Zhao and Yanyan Lou*  \nAbstract \nLung cancer remains the leading cause of cancer-related mortality in both men and women in the US and worldwide. \nNon-small cell lung cancer is the most common variety accounting for 84% of the cases. For a sub

In [9]:
print(response['output'])


Lung cancer is the leading cause of cancer-related mortality in both men and women in the US and worldwide. Non-small cell lung cancer (NSCLC) is the most common variety, accounting for 84% of cases. For a subset of patients with actionable mutations, targeted therapy continues to provide durable responses. Advances in molecular and immunohistochemical techniques have made it possible to usher lung cancer into the era of personalised medicine, with the patient getting individualised treatment based on these markers. 

The approval and adoption of agents targeting these alterations have contributed to the decline in incidence-based mortality from 35% among men with NSCLC diagnosed in 2001 to 26% among those diagnosed in 2014. Similar patterns have been found among women with NSCLC. Despite these new therapeutic options for patients with advanced NSCLC, there continue to be significant challenges as resistance development and disease progression occur in most of these patients. 

Next-ge

In [12]:
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain.memory import ChatMessageHistory

memory = ChatMessageHistory(session_id="test-session")

agent_with_chat_history = RunnableWithMessageHistory(
    agent_executor,
    # This is needed because in most real world scenarios, a session id is needed
    # It isn't really used here because we are using a simple in memory ChatMessageHistory
    lambda session_id: memory,
    input_messages_key="input",
    history_messages_key="chat_history",
)


In [16]:
response = agent_with_chat_history.invoke(
    {"input": "Summarize the main findings of the paper '{}'?".format(title2)},
    config={"configurable": {"session_id": "<foo>"}},
)
print(response['output'])

Parent run 666b5754-b2da-4d11-85fa-8d274462b115 not found for run 8d920876-2e85-4472-962d-d8fd2fc81f92. Treating as a root run.




> Entering new AgentExecutor chain...

I will search for the paper in the database and summarise its main findings.
{'tool_name': 'query_vector_db_articles', 'parameters': {'query': 'Low titers of SARS-CoV-2 neutralizing antibodies after first vaccination dose in cancer patients receiving checkpoint inhibitors', 'metadata': {'title': 'Low titers of SARS-CoV-2 neutralizing antibodies after first vaccination dose in cancer patients receiving checkpoint inhibitors'}}}
[Document(page_content='Terpos\xa0 et\xa0al. J Hematol Oncol           (2021) 14:86  \nhttps://doi.org/10.1186/s13045-021-01099-x\nLETTER TO THE EDITOR\nLow titers of\xa0SARS -CoV -2 neutralizing \nantibodies after\xa0first vaccination dose in\xa0cancer \npatients receiving checkpoint inhibitors\nEvangelos Terpos1* , Flora Zagouri1, Michalis Liontos1, Aimilia D. Sklirou2, Konstantinos Koutsoukos1, \nChristos Markellos1, Alexandros Briasoulis1, Eleni‑Dimitra Papanagnou2, Ioannis P . Trougakos2 and \nMeletios‑Athanasios Dimo

In [17]:
response = agent_with_chat_history.invoke(
    {"input": "What disease are they talking about?"},
    config={"configurable": {"session_id": "<foo>"}},
)

print(response['output'])

Parent run 2f355c80-6d3f-4fc2-bc47-91e9c8f98ae7 not found for run 46766178-ccc3-4a26-bcae-f5e60fe7ff78. Treating as a root run.




> Entering new AgentExecutor chain...

I will read the paper to find out which disease is being referred to.
{'tool_name': 'query_vector_db_articles', 'parameters': {'query': 'Low titers of SARS-CoV-2 neutralizing antibodies after first vaccination dose in cancer patients receiving checkpoint inhibitors.', 'metadata': {'title': 'Low titers of SARS-CoV-2 neutralizing antibodies after first vaccination dose in cancer patients receiving checkpoint inhibitors.'}}}
[Document(page_content='Terpos\xa0 et\xa0al. J Hematol Oncol           (2021) 14:86  \nhttps://doi.org/10.1186/s13045-021-01099-x\nLETTER TO THE EDITOR\nLow titers of\xa0SARS -CoV -2 neutralizing \nantibodies after\xa0first vaccination dose in\xa0cancer \npatients receiving checkpoint inhibitors\nEvangelos Terpos1* , Flora Zagouri1, Michalis Liontos1, Aimilia D. Sklirou2, Konstantinos Koutsoukos1, \nChristos Markellos1, Alexandros Briasoulis1, Eleni‑Dimitra Papanagnou2, Ioannis P . Trougakos2 and \nMeletios‑Athanasios Dimopoulo